In [1]:
import json
import os
import pandas as pd
from pathlib import Path
from urllib.parse import unquote

In [2]:
def extrair_id_animal(nome_imagem):
    # Remove a extensão para facilitar a quebra de strings
    nome = nome_imagem.replace('.jpg', '')
    
    # Caso 1: Padrão RLC (RLC3_00_...) -> ID é o primeiro bloco
    if nome.startswith('RLC'):
        return nome.split('_')[0]
    
    # Caso 2: Padrão Data (20260101_040807_baia16_IPC1) -> ID é o último bloco
    partes = nome.split('_')
    if len(partes) > 1:
        return partes[-1]
    
    return "DESCONHECIDO"

def consolidar_dataset(caminho_raw):
    lista_final = []
    caminho_base = Path(caminho_raw)
    
    for pasta_aluno in caminho_base.iterdir():
        if not pasta_aluno.is_dir():
            continue
            
        caminho_kp = None
        for sub in ['Key_points', 'keypoints', 'Keypoints']:
            if (pasta_aluno / sub).exists():
                caminho_kp = pasta_aluno / sub
                break
        
        if caminho_kp:
            print(f"Processando: {pasta_aluno.name}")
            for arquivo_json in caminho_kp.iterdir():
                if arquivo_json.is_dir() or arquivo_json.name.startswith('.'):
                    continue
                
                try:
                    with open(arquivo_json, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    
                    tasks = data if isinstance(data, list) else [data]
                    
                    for task in tasks:
                        img_path = task.get('task', {}).get('data', {}).get('img', '')
                        nome_imagem = unquote(img_path.split('/')[-1])
                        
                        # NOVA LÓGICA: Extrair o ID aqui na origem
                        animal_id = extrair_id_animal(nome_imagem)
                        
                        results = task.get('result', [])
                        temp_map = {}
                        for res in results:
                            res_id = res['id']
                            if res_id not in temp_map:
                                temp_map[res_id] = {
                                    'aluno': pasta_aluno.name, 
                                    'imagem': nome_imagem,
                                    'animal_id': animal_id # Incluído no dicionário
                                }
                            
                            val = res.get('value', {})
                            if res['type'] == 'keypointlabels':
                                temp_map[res_id]['ponto'] = val.get('keypointlabels', ['unknown'])[0]
                                temp_map[res_id]['x'] = val.get('x')
                                temp_map[res_id]['y'] = val.get('y')
                            elif res['type'] == 'choices':
                                temp_map[res_id]['visibilidade'] = val.get('choices', [None])[0]

                        for item in temp_map.values():
                            if 'ponto' in item and item['ponto'] != 'cow':
                                lista_final.append(item)
                except Exception as e:
                    print(f"Erro ao ler {arquivo_json}: {e}")
    
    return pd.DataFrame(lista_final)

In [3]:
# Execução
df = consolidar_dataset('../data/raw')

if not df.empty:
    # Garante que a pasta processed existe
    Path('../data/processed').mkdir(parents=True, exist_ok=True)
    df.to_csv('../data/processed/dataset_completo.csv', index=False)
    print(f"\nSucesso! {len(df)} pontos consolidados.")
else:
    print("\nO DataFrame continua vazio. Verifique se os arquivos dentro das pastas são realmente JSONs válidos.")

df.head(10)

Processando: 13 - Juliano - Complemento (pronto)
Processando: 08 - Eduardo Braga (pronto)
Processando: 14 - Lucas (pronto)
Processando: 05 - Camilla (pronto)
Processando: 06 - Carlos Augusto (pronto)
Processando: 15 - Marcelo Azevedo (pronto)
Processando: 23 - Ronen (pronto)
Processando: 17 - Miguel (pronto)
Processando: 21 - Raquel (pronto)
Processando: 07 - Carlos Henrique (pronto)
Processando: 11 - Gustavo (pronto)
Processando: 03 - Raony (pronto)
Processando: 01 - Thales (pronto)
Processando: 20 - Rafael (pronto)
Processando: 12 - Israel (pronto)
Processando: 13 - Juliano (pronto)
Processando: 22 - Renato (pronto)
Processando: 04 - Alex (pronto)
Processando: 19 - Paulo (pronto)

Sucesso! 6868 pontos consolidados.


,aluno,imagem,animal_id,ponto,x,y,visibilidade
0,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,withers,10.714656,49.504950,Visível
1,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,back,39.367333,49.834983,Visível
2,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,hook up,67.658841,10.231023,Visível
3,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,hook down,69.946240,85.148515,Visível
4,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,hip,71.752081,48.184818,Visível
5,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,tail head,85.837640,47.524752,Visível
6,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,pin up,92.218278,27.722772,Visível
7,13 - Juliano - Complemento (pronto),RLC4_00_20260114221438_baia10_RLC4.jpg,RLC4,pin down,91.616331,70.957096,Visível
8,13 - Juliano - Complemento (pronto),RLC4_00_20260114221037_baia1_RLC4.jpg,RLC4,withers,8.784597,32.819787,Visível
9,13 - Juliano - Complemento (pronto),RLC4_00_20260114221037_baia1_RLC4.jpg,RLC4,back,35.138387,34.164860,Visível
